In [ ]:
def netCDF4_to_csv(nc4_key, nc4_url, yearN=1980): #Condition- function starts from year 1980, nc4_file_path -> str ,nc4_key -> str, datatype either monthly or yearly
    try:
        import requests
        import h5py
        import netCDF4
        from netCDF4 import Dataset
        import pandas as pd
        import numpy as np
        import datetime as dateformat
    except:
        print("Error - Required Libraries not installed")
        sys.exit()
    
    try:
        r = requests.get(nc4_url, allow_redirects=True)
        open(nc4_key+'.nc4', 'wb').write(r.content)
        ds = Dataset(nc4_key+'.nc4')
    except:
        print('Error - Dataset not able to find the file')
        sys.exit()
    
    
    if type(nc4_key) is not str:
        print('Error - Check input for key type')
        sys.exit()
        
    if type(yearN) is not int:
        print('Error - Check input for year')
        sys.exit() 
    elif yearN < 0:
        print("Error- enter a positive number for year")
        sys.exit()
        
    
    ### This will determine the number of years/months in the NC4 data   
    f = h5py.File(nc4_key+'.nc4', 'r')
    print(f.keys())
    size = f[nc4_key].shape
    num = size[0]
    
    key = ds[nc4_key][:]

    lat = ds['lat'][:]
    lon = ds['lon'][:]
    
    year = yearN
    count = 0
    month = 1
    
    
    # monthly data
    if num == 1392:
        start = (1900 - year - 1)*12;
        datatype = 'Monthly_Output'
        print(datatype)
        
        #Jan of YearN to Dec of 2016
        for i in range(start,num):
            data = key[i,:,:]
            if i == start:
                data_pd = pd.DataFrame(data = data)
                lat = ds['lat'][:]
                data_pd.index =  lat
                lon = ds['lon'][:]
                data_pd.columns = lon
                data_pd = pd.melt(data_pd, ignore_index = False, var_name='lon', value_name=str(dateformat.datetime(year, 1, month).strftime('%b/%Y')))
            else:
                data_I = pd.DataFrame(data = data)
                data_I.index =  lat
                
                if i % 12 == 0:
                    year = year+1
                    month = 1
                else:
                    month = month + 1
                    
                data_I = pd.melt(data_I,ignore_index = False, value_name= str(dateformat.datetime(year, month,1).strftime('%b/%Y')))
                data_pd[str(dateformat.datetime(year,month,1).strftime('%b/%Y'))] = data_I.loc[:,str(dateformat.datetime(year,month,1).strftime('%b/%Y'))]
               
        
    # yearly data
    elif num == 116:
        start = 1900 - year;
        datatype = 'Yearly_Output'
        print(datatype)
        
        #YearN to 2016
        for i in range(start,num):
            data = key[i,:,:]
            if i == start:
                data_pd = pd.DataFrame(data = data)
                lat = ds['lat'][:]
                data_pd.index =  lat
                lon = ds['lon'][:]
                data_pd.columns = lon
                data_pd = pd.melt(data_pd, ignore_index = False, var_name='lon', value_name=str(yearN))
            else:
                data_I = pd.DataFrame(data = data)
                data_I.index =  lat
                year = year+1
                data_I = pd.melt(data_I,ignore_index = False, value_name= str(year))
                data_pd[str(year)] = data_I.loc[:,str(year)]
            
    data_pd.to_csv("watergap_"+nc4_key+"_"+datatype+".csv")
    return data_pd
    

In [ ]:
netCDF4_to_csv(nc4_key='pdomuse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pdomuse_yearly_1901_2016.nc4')

In [ ]:
netCDF4_to_csv(nc4_key='pirruse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pirruse_monthly_1901_2016.nc4')

In [ ]:
netCDF4_to_csv(nc4_key='pinduse', nc4_url='https://hs.pangaea.de/model/WaterGAP_v2-2d/watergap_22d_WFDEI-GPCC_histsoc_pinduse_yearly_1901_2016.nc4')